# POP MOC(sig2) for 0.1-degree
 - Computes MOC(sig2) offline from POP history files using simple xhistogram binning

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import dask
from xhistogram.xarray import histogram
import pop_tools
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline

from MOCutils import popmoc

In [2]:
import dask
from dask.distributed import wait
dask.__version__

'2022.11.0'

### Start Dask Cluster

In [16]:
# Close out Dask Cluster and release workers:
client.shutdown()

In [3]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=20GB',
        account='NCGD0011',
        walltime='06:00:00',
        local_directory='/glade/scratch/yeager/',
        log_directory='/glade/scratch/yeager/',
        interface='ib0',
        job_script_prologue=[
        f'export ESMFMKFILE={os.environ["ESMFMKFILE"]}'])

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        "distributed.scheduler.worker-saturation": 1.0,
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

In [4]:
cluster, client = get_ClusterClient()
cluster.scale(60) 

/glade/work/yeager/conda-envs/mypythonenv/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38040 instead
  warnings.warn(


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/38040/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/38040/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.34:40849,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/yeager/proxy/38040/status,Total threads: 0
Started: Just now,Total memory: 0 B


### 1. Read in Needed Grid Info

In [6]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    #ds_out = ds.copy(deep)
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

def preprocessor(ds0):
    keepvars = ['UVEL','VVEL','TEMP','SALT']
    d0 = ds0[keepvars]
    d0 = time_set_midmonth(d0,'time')
    #d0 = d0.groupby('time.year').mean('time').rename({'year':'time'})
    return d0

In [7]:
fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

In [8]:
%%time
dz = ds_grid['dz'].persist() / 100.
kmt = ds_grid['KMT'].fillna(0).persist() 
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)

CPU times: user 15.2 s, sys: 14.9 s, total: 30.1 s
Wall time: 32.4 s


In [9]:
tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

In [10]:
# Compute sigma-2 field from POP model output
refz = 2000
refdep = xr.DataArray(refz)

# Grid Metrics
dxu = ds_grid['DXU']
dyu = ds_grid['DYU']
dxt = ds_grid['DXT']
dyt = ds_grid['DYT']

### MOC Region Mask

In [11]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [12]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

### Sigma Target Grid

In [13]:
# Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

# Loop over years and compute MOC 

In [14]:
ystart=89
yend=103
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/'
fin_tmp = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVV.YYYY01-YYYY12.nc'
fin_tmp_out = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOCsig.YYYY.nc'
outdir = '/glade/scratch/yeager/BSPNA'

In [14]:
ystart=104
yend=115
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/'
fin_tmp = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVV.YYYY01-YYYY12.nc'
fin_tmp_out = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOCsig.YYYY.nc'
outdir = '/glade/scratch/yeager/BSPNA'

In [14]:
ystart=119
yend=130
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/'
fin_tmp = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVV.YYYY01-YYYY12.nc'
fin_tmp_out = 'B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOCsig.YYYY.nc'
outdir = '/glade/scratch/yeager/BSPNA'

In [15]:
%%time
for n in np.arange(ystart,yend+1,1):
    yrstr = str(n).zfill(4)
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','VVEL')
    print("working on {}".format(fin))
    dsV = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','UVEL')
    dsU = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','TEMP')
    dsT = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    fin = fdir+fin_tmp.replace('YYYY',yrstr).replace('VVV','SALT')
    dsS = time_set_midmonth(xr.open_dataset(fin, chunks={'time':1,'nlon':100}),'time')
    
    
    u_e = dsU['UVEL']
    u_e = u_e.where(u_e<1.e30,0)
    v_e = dsV['VVEL']
    v_e = v_e.where(v_e<1.e30,0)
    salt = dsS['SALT']
    temp = dsT['TEMP']

    # Sigma2 on model TLAT, TLONG
    sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
    sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

    # Grid-oriented Volume FLuxes:
    u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})

    # Convert u_e,v_e to C-grid fluxes
    u = 0.5*(u_e+u_e.shift(nlat=1))
    v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

    # Volume fluxes in density-space. 
    iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
    iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
    iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

    # Compute Vertical Volume Flux from horizontal flux convergence
    wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
    wflux = wflux.assign_coords({'TLAT':tlat,'TLONG':tlon})

    # Compute MOC
    MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
    MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
    MOC = dask.optimize(MOC)[0]

    # add vflux at southern boundary of Atlantic domain
    tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
    tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
    moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
    moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
    MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

    # Save to netcdf
    MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
    dsout = MOCann.to_dataset()

    fout = fin_tmp_out.replace('YYYY',yrstr)
    fout = os.path.join(outdir,fout)
    dsout.to_netcdf(fout,unlimited_dims='time')
    print("wrote {} at {}".format(fout, time.ctime()))

working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.011901-011912.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOCsig.0119.nc at Sun Oct  1 07:19:18 2023
working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.012001-012012.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.MOCsig.0120.nc at Sun Oct  1 07:24:10 2023
working on /glade/campaign/collections/cmip/CMIP6/iHESP/HighResMIP/B1950cntl/HR/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013/ocn/proc/tseries/month_1/B.E.13.B1950C5.ne120_t12.cesm-ihesp-1950cntl.013.pop.h.VVEL.012101-012112.nc
wrote /glade/scratch/yeager/BSPNA/B.E.13.B1